In [9]:
import numpy as np
import tqdm
import torch
from sklearn.linear_model import SGDClassifier
import time
from torch.optim import SGD, Adam
import random
import sklearn

/home/curttigges/miniconda3/envs/rlace/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
EVAL_CLF_PARAMS = {
    "loss": "log",
    "tol": 1e-4,
    "iters_no_change": 15,
    "alpha": 1e-4,
    "max_iter": 25000,
}
NUM_CLFS_IN_EVAL = 3  # change to 1 for large dataset / high dimensionality

In [11]:
def init_classifier():

    return SGDClassifier(
        loss=EVAL_CLF_PARAMS["loss"],
        fit_intercept=True,
        max_iter=EVAL_CLF_PARAMS["max_iter"],
        tol=EVAL_CLF_PARAMS["tol"],
        n_iter_no_change=EVAL_CLF_PARAMS["iters_no_change"],
        n_jobs=32,
        alpha=EVAL_CLF_PARAMS["alpha"],
    )

In [12]:
def symmetric(X):
    """Make a matrix symmetric by averaging with its transpose."""
    X.data = 0.5 * (X.data + X.data.T)
    return X

In [13]:
def get_score(X_train, y_train, X_dev, y_dev, P, rank):
    """Get the score of a projection matrix P on the dev set."""
    P_svd = get_projection(P, rank)

    loss_vals = []
    accs = []

    for i in range(NUM_CLFS_IN_EVAL):
        clf = init_classifier()
        clf.fit(X_train @ P_svd, y_train)
        y_pred = clf.predict_proba(X_dev @ P_svd)
        loss = sklearn.metrics.log_loss(y_dev, y_pred)
        loss_vals.append(loss)
        accs.append(clf.score(X_dev @ P_svd, y_dev))

    i = np.argmin(loss_vals)
    return loss_vals[i], accs[i]

In [14]:
def solve_constraint(lambdas, d=1):
    """Solve the constraint sum(min(max(lambdas - theta, 0), 1)) = d for theta."""

    def f(theta):
        return_val = np.sum(np.minimum(np.maximum(lambdas - theta, 0), 1)) - d
        return return_val

    theta_min, theta_max = max(lambdas), min(lambdas) - 1
    assert f(theta_min) * f(theta_max) < 0

    mid = (theta_min + theta_max) / 2
    tol = 1e-4
    iters = 0

    while iters < 25:

        mid = (theta_min + theta_max) / 2

        if f(mid) * f(theta_min) > 0:

            theta_min = mid
        else:
            theta_max = mid
        iters += 1

    lambdas_plus = np.minimum(np.maximum(lambdas - mid, 0), 1)
    # if (theta_min-theta_max)**2 > tol:
    #    print("didn't converge", (theta_min-theta_max)**2)
    return lambdas_plus

In [15]:
def get_majority_acc(y):
    """Get the majority accuracy of a set of labels."""
    from collections import Counter

    c = Counter(y)
    fracts = [v / sum(c.values()) for v in c.values()]
    maj = max(fracts)
    return maj

In [16]:
def get_entropy(y):
    """Get the entropy of a set of labels."""

    from collections import Counter
    import scipy

    c = Counter(y)
    fracts = [v / sum(c.values()) for v in c.values()]
    return scipy.stats.entropy(fracts)

Makes the projection matrix symmetric.
![symmetric](images/symmetric.png)

In [17]:
def symmetric(X):
    """Make a matrix symmetric by averaging with its transpose."""
    X.data = 0.5 * (X.data + X.data.T)
    return X

Creates an orthogonal projection matrix as specified below:
![SVD for projection matrix](images/svd-projection-step.png)

In [18]:
def get_projection(P, rank):
    """Get the projection matrix from the SVD of P."""
    D, U = np.linalg.eigh(P)
    U = U.T
    W = U[-rank:]
    P_final = np.eye(P.shape[0]) - W.T @ W
    return P_final

Prints output before and after SVD.

In [19]:
def prepare_output(P, rank, score):
    """Prepare the output of the algorithm."""
    P_final = get_projection(P, rank)
    return {"score": score, "P_before_svd": np.eye(P.shape[0]) - P, "P": P_final}

The method below runs the inner and outer loops of the RLACE algorithm, as specified in the accompanying image.
![Adversarial game](images/run-adversarial-game.png)

In [20]:
def solve_adv_game(
    X_train,
    y_train,
    X_dev,
    y_dev,
    rank=1,
    device="cpu",
    out_iters=75000,
    in_iters_adv=1,
    in_iters_clf=1,
    epsilon=0.0015,
    batch_size=128,
    evalaute_every=1000,
    optimizer_class=SGD,
    optimizer_params_P={"lr": 0.005, "weight_decay": 1e-4},
    optimizer_params_predictor={"lr": 0.005, "weight_decay": 1e-4},
):
    """
    Run the adversarial game.

    Args:
        X_train: The training data
        y_train: The training labels
        X_dev: The dev data
        y_dev: The dev labels
        rank: The rank of the projection matrix
        device: The device to run on
        out_iters: The number of outer iterations
        in_iters_adv: The number of inner iterations for the adversary
        in_iters_clf: The number of inner iterations for the classifier
        epsilon: The epsilon for the adversary
        batch_size: The batch size
        evalaute_every: Evaluate every n iterations
        optimizer_class: The optimizer class
        optimizer_params_P: The optimizer parameters for the projection matrix
        optimizer_params_predictor: The optimizer parameters for the predictor

    Returns:
        The projection matrix
    """

    def get_loss_fn(X, y, predictor, P, bce_loss_fn, optimize_P=False):
        """Get the loss function.

        Args:
            X: The input
            y: The labels
            predictor: The predictor
            P: The projection matrix
            bce_loss_fn: The loss function
            optimize_P: Whether to optimize the projection matrix or not. If True, the loss is negated.

        Returns:
            The loss
        """
        I = torch.eye(X_train.shape[1]).to(device)
        bce = bce_loss_fn(predictor(X @ (I - P)).squeeze(), y)
        if optimize_P:
            bce = -bce
        return bce

    X_torch = torch.tensor(X_train).float().to(device)
    y_torch = torch.tensor(y_train).float().to(device)

    num_labels = len(set(y_train.tolist()))

    # set the loss function based on the number of labels
    if num_labels == 2:
        predictor = torch.nn.Linear(X_train.shape[1], 1).to(device)
        bce_loss_fn = torch.nn.BCEWithLogitsLoss()
        y_torch = y_torch.float()
    else:
        predictor = torch.nn.Linear(X_train.shape[1], num_labels).to(device)
        bce_loss_fn = torch.nn.CrossEntropyLoss()
        y_torch = y_torch.long()

    # initialize the projection matrix
    P = 1e-1 * torch.randn(X_train.shape[1], X_train.shape[1]).to(device)
    P.requires_grad = True

    # initialize the optimizers
    optimizer_predictor = optimizer_class(
        predictor.parameters(), **optimizer_params_predictor
    )
    optimizer_P = optimizer_class([P], **optimizer_params_P)

    # get the majority accuracy and entropy of the training set
    maj = get_majority_acc(y_train)
    label_entropy = get_entropy(y_train)
    pbar = tqdm.tqdm(range(out_iters), total=out_iters, ascii=True)
    count_examples = 0
    best_P, best_score, best_loss = None, 1, -1

    # run the outer loop
    for i in pbar:

        for j in range(in_iters_adv):
            # optimize the projection matrix
            # this matches up with the second inner loop of the algorithm in the paper
            P = symmetric(P)
            optimizer_P.zero_grad()

            idx = np.arange(0, X_torch.shape[0])
            np.random.shuffle(idx)
            X_batch, y_batch = X_torch[idx[:batch_size]], y_torch[idx[:batch_size]]

            loss_P = get_loss_fn(
                X_batch, y_batch, predictor, symmetric(P), bce_loss_fn, optimize_P=True
            )
            loss_P.backward()
            optimizer_P.step()

            # project

            with torch.no_grad():
                # project the projection matrix on the Fantope using lemma B.1
                D, U = torch.linalg.eigh(symmetric(P).detach().cpu())
                D = D.detach().cpu().numpy()
                D_plus_diag = solve_constraint(D, d=rank)
                D = torch.tensor(np.diag(D_plus_diag).real).float().to(device)
                U = U.to(device)
                P.data = U @ D @ U.T

        for j in range(in_iters_clf):
            # optimize the predictor
            # this matches up with the first inner loop of the algorithm in the paper
            optimizer_predictor.zero_grad()
            idx = np.arange(0, X_torch.shape[0])
            np.random.shuffle(idx)
            X_batch, y_batch = X_torch[idx[:batch_size]], y_torch[idx[:batch_size]]

            loss_predictor = get_loss_fn(
                X_batch, y_batch, predictor, symmetric(P), bce_loss_fn, optimize_P=False
            )
            loss_predictor.backward()
            optimizer_predictor.step()
            count_examples += batch_size

        if i % evalaute_every == 0:
            # pbar.set_description("Evaluating current adversary...")
            loss_val, score = get_score(
                X_train, y_train, X_train, y_train, P.detach().cpu().numpy(), rank
            )
            if (
                loss_val > best_loss
            ):  # if np.abs(score - maj) < np.abs(best_score - maj):
                best_P, best_loss = symmetric(P).detach().cpu().numpy().copy(), loss_val
            if np.abs(score - maj) < np.abs(best_score - maj):
                best_score = score

            # update progress bar

            best_so_far = (
                best_score if np.abs(best_score - maj) < np.abs(score - maj) else score
            )

            pbar.set_description(
                "{:.0f}/{:.0f}. Acc post-projection: {:.3f}%; best so-far: {:.3f}%; Maj: {:.3f}%; Gap: {:.3f}%; best loss: {:.4f}; current loss: {:.4f}".format(
                    i,
                    out_iters,
                    score * 100,
                    best_so_far * 100,
                    maj * 100,
                    np.abs(best_so_far - maj) * 100,
                    best_loss,
                    loss_val,
                )
            )
            pbar.refresh()  # to show immediately the update
            time.sleep(0.01)

        if i > 1 and np.abs(best_score - maj) < epsilon:
            # if i > 1 and np.abs(best_loss - label_entropy) < epsilon:
            break
    output = prepare_output(best_P, rank, best_score)
    return output

In [21]:
if __name__ == "__main__":

    # random.seed(0)
    # np.random.seed(0)

    # create a synthetic dataset
    n, dim = 15000, 200
    num_classes = 2

    X = np.random.randn(n, dim)
    y = np.random.randint(
        low=0, high=num_classes, size=n
    )  # (np.random.rand(n) > 0.5).astype(int)

    X[:, 0] = (y + np.random.randn(*y.shape) * 0.3) ** 2 + 0.3 * y
    X[:, 1] = (y + np.random.randn(*y.shape) * 0.1) ** 2 - 0.7 * y
    X[:, 2] = (
        (y + np.random.randn(*y.shape) * 0.3) ** 2
        + 0.5 * y
        + np.random.randn(*y.shape) * 0.2
    )
    X[:, 3] = (
        (y + np.random.randn(*y.shape) * 0.5) ** 2
        - 0.7 * y
        + np.random.randn(*y.shape) * 0.1
    )
    X[:, 4] = (
        (y + np.random.randn(*y.shape) * 0.5) ** 2
        - 0.8 * y
        + np.random.randn(*y.shape) * 0.1
    )
    X[:, 5] = (
        (y + np.random.randn(*y.shape) * 0.25) ** 2
        - 0.2 * y
        + np.random.randn(*y.shape) * 0.1
    )
    mixing_matrix = 1e-2 * np.random.randn(dim, dim)
    X = X @ mixing_matrix

    l_train = int(0.6 * n)
    X_train, y_train = X[:l_train], y[:l_train]
    X_dev, y_dev = X[l_train:], y[l_train:]

    # arguments
    num_iters = 50000
    rank = 1
    optimizer_class = torch.optim.SGD
    optimizer_params_P = {"lr": 0.003, "weight_decay": 1e-4}
    optimizer_params_predictor = {"lr": 0.003, "weight_decay": 1e-4}
    epsilon = 0.001  # stop 0.1% from majority acc
    batch_size = 256

    output = solve_adv_game(
        X_train,
        y_train,
        X_dev,
        y_dev,
        rank=rank,
        device="cpu",
        out_iters=num_iters,
        optimizer_class=optimizer_class,
        optimizer_params_P=optimizer_params_P,
        optimizer_params_predictor=optimizer_params_predictor,
        epsilon=epsilon,
        batch_size=batch_size,
    )

    # train a classifier

    P_svd = output["P"]
    P_before_svd = output["P_before_svd"]
    svm = init_classifier()

    svm.fit(X_train[:], y_train[:])
    score_original = svm.score(X_dev, y_dev)

    svm = init_classifier()
    svm.fit(X_train[:] @ P_before_svd, y_train[:])
    score_projected_no_svd = svm.score(X_dev @ P_before_svd, y_dev)

    svm = init_classifier()
    svm.fit(X_train[:] @ P_svd, y_train[:])
    score_projected_svd_dev = svm.score(X_dev @ P_svd, y_dev)
    score_projected_svd_train = svm.score(X_train @ P_svd, y_train)
    maj_acc_dev = get_majority_acc(y_dev)
    maj_acc_train = get_majority_acc(y_train)

    print("===================================================")
    print(
        "Original Acc, dev: {:.3f}%; Acc, projected, no svd, dev: {:.3f}%; Acc, projected+SVD, train: {:.3f}%; Acc, projected+SVD, dev: {:.3f}%".format(
            score_original * 100,
            score_projected_no_svd * 100,
            score_projected_svd_train * 100,
            score_projected_svd_dev * 100,
        )
    )
    print("Majority Acc, dev: {:.3f} %".format(maj_acc_dev * 100))
    print("Majority Acc, train: {:.3f} %".format(maj_acc_train * 100))
    print(
        "Gap, dev: {:.3f} %".format(np.abs(maj_acc_dev - score_projected_svd_dev) * 100)
    )
    print(
        "Gap, train: {:.3f} %".format(
            np.abs(maj_acc_train - score_projected_svd_train) * 100
        )
    )
    print("===================================================")
    eigs_before_svd, _ = np.linalg.eigh(P_before_svd)
    print("Eigenvalues, before SVD: {}".format(eigs_before_svd[:]))

    eigs_after_svd, _ = np.linalg.eigh(P_svd)
    print("Eigenvalues, after SVD: {}".format(eigs_after_svd[:]))

    eps = 1e-6
    assert np.abs((eigs_after_svd > eps).sum() - (dim - rank)) < eps

  0%|          | 0/50000 [00:00<?, ?it/s]/home/curttigges/miniconda3/envs/rlace/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/curttigges/miniconda3/envs/rlace/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
/home/curttigges/miniconda3/envs/rlace/lib/python3.8/site-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
0/50000. Acc post-projection: 96.633%; best so-far: 96.633%; Maj: 50.822%; Gap: 45.811%; best loss: 0.1272; current loss: 0.1272:   2%|1         | 983/50000 [00:

Original Acc, dev: 96.567%; Acc, projected, no svd, dev: 49.917%; Acc, projected+SVD, train: 50.833%; Acc, projected+SVD, dev: 49.950%
Majority Acc, dev: 50.017 %
Majority Acc, train: 50.822 %
Gap, dev: 0.067 %
Gap, train: 0.011 %
Eigenvalues, before SVD: [6.79300139e-07 9.99999993e-01 9.99999994e-01 9.99999995e-01
 9.99999995e-01 9.99999995e-01 9.99999996e-01 9.99999996e-01
 9.99999996e-01 9.99999996e-01 9.99999996e-01 9.99999997e-01
 9.99999997e-01 9.99999997e-01 9.99999997e-01 9.99999997e-01
 9.99999997e-01 9.99999998e-01 9.99999998e-01 9.99999998e-01
 9.99999998e-01 9.99999998e-01 9.99999998e-01 9.99999998e-01
 9.99999998e-01 9.99999998e-01 9.99999998e-01 9.99999998e-01
 9.99999998e-01 9.99999999e-01 9.99999999e-01 9.99999999e-01
 9.99999999e-01 9.99999999e-01 9.99999999e-01 9.99999999e-01
 9.99999999e-01 9.99999999e-01 9.99999999e-01 9.99999999e-01
 9.99999999e-01 9.99999999e-01 9.99999999e-01 9.99999999e-01
 9.99999999e-01 9.99999999e-01 9.99999999e-01 9.99999999e-01
 9.99999999e